# Tutorial 2. Network Generation, part 2

In this tutorial we explore the different types of network orderings that ```network_layout()``` function from the *netgen* module can generate.

### Imports

In [1]:
import geopandas as gpd
import pandas as pd
import netsim.netgen as ng

### Read a sample shapefile

In [2]:
fn = r'..\data\sample\sample5.shp'

We use geopandas to read the shapefile that contains the location and the columns needed to run the simulation. In case the location file is a simple a text file, e.g. *comma-delimited* or *csv*, use
```python
import pandas as pd
df = pd.read_csv(filename)
```
instead.


In [3]:
df= gpd.read_file(fn)
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,1,0,530403,4388580,POINT (530403 4388580)
3,3,1,1,0,530503,4388620,POINT (530503 4388620)
4,4,1,1,0,530729,4388930,POINT (530729 4388930)
5,5,1,1,0,530606,4389150,POINT (530606 4389150)


Let's change the number of groups so that we end up with two groups of two and four locations.

In [4]:
sel = df['id'].isin([2, 3, 4, 5])
df.loc[sel,'group']= 2
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,2,0,530403,4388580,POINT (530403 4388580)
3,3,1,2,0,530503,4388620,POINT (530503 4388620)
4,4,1,2,0,530729,4388930,POINT (530729 4388930)
5,5,1,2,0,530606,4389150,POINT (530606 4389150)


We run the ```setup()``` function in the *netgen* module to check whether the table with the information needed to run the simulation has all the appropriate columns and values within these. 

In [5]:
df = ng.setup(df)


 No corrections or errors !! 


### Creating a network generator

The next step we run the ```create_network_generator()``` function to create a *network generator* that we can  use to produce different versions, or iterations, of our network.

In [6]:
netgentor, net_info, total_iterations = ng.create_network_generator(df)


 iteration broken per group....

   group    iter_type  num_iter  num_loc
0      1  permutation         2        2
1      2  permutation        24        4

 total number of iterations.... 48


### Creating a network of paths

 The ```network_layout()``` function transforms an iteration generated by the generator created using the  ```create_network_generator()``` function into a network of paths. This function can generate several different types of networks. Let's explore this using a single iteration,

In [7]:
sample_iteration = list(next(netgentor))
sample_iteration

[(0, 1), (2, 3, 4, 5)]

Let's generate the a 'close' network of paths, the *default* network type. This type of network is defined as a close circuit of paths amongst the different locations in each group. Each group is independent (no connection) from each other

In [8]:
df_net = ng.network_layout(df, sample_iteration, 1)
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,3,1
3,3,4,1
4,4,5,1
5,5,2,1


Note that with the exception of the first group (which is only made of two locations), there is only one path for each pair of locations. If we want, we can change this by including the parameter ```twoway = True``` when calling the function (this parameter can be used with all network options) 

In [9]:
df_net = ng.network_layout(df, sample_iteration, 1, twoway= True)
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,3,1
3,3,2,1
4,3,4,1
5,4,3,1
6,4,5,1
7,5,4,1
8,5,2,1
9,2,5,1


With the *opt* parameter  ```network_layout()``` can define other types of network. Currently *opt* can take the following values ```['close', 'central', 'decentral', 'distributed', 'all]```

##### *central*ized network

In [10]:
df_net = ng.network_layout(df, sample_iteration, 1, opt='central')
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,0,1
3,2,1,1
4,3,0,1
5,3,1,1
6,4,0,1
7,4,1,1
8,5,0,1
9,5,1,1


This option defines a network of paths that consists of a centralized set of paths from the locations of the first group to all of the locations in the remaining groups. Paths amongst the locations in the first group are generated first and then paths from the remaining locations (regardless of whether they are in different groups) to each of the first group locations are generated.

##### *decentral*ized network

In [11]:
df_net = ng.network_layout(df, sample_iteration, 1, opt='decentral')
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,0,1
3,2,1,1
4,3,0,1
5,3,1,1
6,4,0,1
7,4,1,1
8,5,0,1
9,5,1,1


This option defines a network of paths so that the locations of each group are connected to the locations of the following (lower level) group.

##### *distributed* network

In [12]:
df_net = ng.network_layout(df, sample_iteration, 1, opt='distributed')
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,0,1
3,2,1,1
4,3,0,1
5,3,1,1
6,4,0,1
7,4,1,1
8,5,0,1
9,5,1,1


This option defines a network of paths similar to *decentral*, where the locations of each group are connected to the locations of the following (lower level) group. However, in addition, locations within each group are connected amongst themselves.

##### *all* network

In [13]:
df_net = ng.network_layout(df, sample_iteration, 1, opt='all')
df_net

,origin,destination,iteration
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
5,1,2,1
6,1,3,1
7,1,4,1
8,1,5,1
9,2,3,1


This option defines a network of paths from all to all locations regardless of whether they are from different groups.

Finally, it is possible to define a network for more than one iteration. In order to do this, however, we need to provide ```network_layout()``` function with a blank dataframe. This is shown in the following cell,

In [19]:
# create a blank dataframe
net = {'origin': 'int32', 'destination': 'int32', 'iteration': 'int32'}
df_net = pd.DataFrame(columns=list(net.keys())).astype(net)

for i in range(3):
    iteration = list(next(netgentor))
    print(iteration)
    df_net = ng.network_layout(df, iteration, i, df_net= df_net, opt='close')
df_net

[(0, 1), (3, 5, 2, 4)]
[(0, 1), (3, 5, 4, 2)]
[(0, 1), (4, 2, 3, 5)]


,origin,destination,iteration
0,0,1,0
1,1,0,0
2,2,3,0
3,3,4,0
4,4,5,0
5,5,2,0
6,0,1,1
7,1,0,1
8,2,3,1
9,3,4,1
